In [11]:
# Libaries
from selenium import webdriver
from selenium.common import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json  # Import the JSON module
import pymongo

In [12]:
# Database connection
""" in case you want to create your on database, just change this 3 variables according to your database. 
Database and collections will be created automatically."""
username = "globaladmin"
password = "UEKaeSc8Q3LK5naw"
db_url = "cluster1.etdy2wm.mongodb.net"

# database string
CNX_STR = f"mongodb+srv://{username}:{password}@{db_url}/?connectTimeoutMS=50000"
client = pymongo.MongoClient(CNX_STR)

# heck if the database exists
if "manage2sail" in client.list_database_names():
    print("Database exists!")
    db = client.manage2sail
else:
    print("Database does not exist!")

Database exists!


In [19]:
def scrape_eventdetails(driver):
    event_details_dict = {}
    selector = '#details > table'
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, selector)))
    table = driver.find_element(By.CSS_SELECTOR, selector)
    for row in table.find_elements(By.TAG_NAME, 'tr'):
        columns = row.find_elements(By.TAG_NAME, 'td')
        if len(columns) >= 2:
            key = columns[0].text.strip()  # Clean key from any whitespace
            value = columns[1].text.strip()  # Clean value from any whitespace
            event_details_dict[key] = value  # Store in dictionary using key-value pairing
    return event_details_dict

In [14]:
# function to scrape the event results
def scrape_eventresults(base_url):
    results = []  # Initialize a list to store the results
    try:
        results_url = base_url + '/results'
        driver.get(results_url)
    except:
        print("no resultspage found")
# Wait for the dropdown to be clickable and select it
    try:
        dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#results > div > div > select')))
        select = Select(dropdown)

        # Iterate through all options in the dropdown
        for index in range(len(select.options)):
            # Refresh the dropdown and select object each iteration
            dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#results > div > div > select')))
            select = Select(dropdown)
            select.select_by_index(index)
            time.sleep(5)  # Adjust the sleep time if necessary

            # Find and process the table data
            selector = '#results > div > div > div:nth-child(3) > div:nth-child(5) > div:nth-child(2) > div:nth-child(3) > table:nth-child(4)'
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, selector)))
            table = driver.find_element(By.CSS_SELECTOR, selector)
            for row in table.find_elements(By.TAG_NAME, 'tr'):
                columns = row.find_elements(By.TAG_NAME, 'td')
                row_data = {col.text for i, col in enumerate(columns)}
                results.append(row_data)  # Append each row to results list

    except:
        selector = '#results > div > div > div:nth-child(3) > div:nth-child(5) > div:nth-child(2) > div:nth-child(3) > table:nth-child(4)'
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, selector)))
        table = driver.find_element(By.CSS_SELECTOR, selector)
        for row in table.find_elements(By.TAG_NAME, 'tr'):
            columns = row.find_elements(By.TAG_NAME, 'td')
            row_data = {f"column_{i}": col.text for i, col in enumerate(columns)}
            results.append(row_data)  # Append each row to results list

    return results

In [23]:
if __name__ == "__main__":
    # Setup WebDriver
    options = Options()
    service = Service('/usr/local/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=options)
    
    collection = db.events
    try:
        for document in collection.find().limit(3):
            url = document['link']
            driver.get(url)
            event_details = scrape_eventdetails(driver)
            print(event_details)
            update_result = collection.update_one(
                {'_id': document['_id']},
                {'$set': {'eventdetails': event_details}}  # event_details is now a dict
            )
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()





{'Anmeldung :': '22.03.2023 - 22.04.2023', 'Veranstaltung :': '22.04.2023 - 23.04.2023 Add to calendar', 'Internet :': 'www.scap.ch', 'eMail :': 'kontakt@scap.ch', 'Verein :': 'Segelclub am Pfäffikersee', 'Ort :': 'Slipanlage Auslikon', 'Adresse :': 'Pfäffikersee', 'Stadt :': '', 'Postleitzahl :': '', 'Land :': 'Switzerland', 'Id :': '93664154-3029-4003-ba86-8f26379cd1ef'}
{'Anmeldung :': '01.01.2023 - 28.04.2023', 'Veranstaltung :': '29.04.2023 - 30.04.2023 Add to calendar', 'Internet :': 'regatta@scoz.ch', 'eMail :': 'regatta@scoz.ch', 'Telefon :': '+41 799361955', 'Verein :': 'Segelclub Oberer Zürichsee', 'Ort :': 'Hafen Kiebitz', 'Adresse :': 'Bruggholzstrasse 8a', 'Stadt :': 'Nuolen', 'Postleitzahl :': '8855', 'Land :': 'Switzerland', 'Id :': '9a6eaa67-cb75-4ef2-97d9-41826f26c28c'}
{'Anmeldung :': '01.01.2023 - 29.04.2023', 'Veranstaltung :': '29.04.2023 - 30.04.2023 Add to calendar', 'Internet :': 'www.z-s-c.ch', 'eMail :': 'regatta@z-s-c.ch', 'Telefon :': '+41 79 248 40 79', 'Ve